### Imports

In [1]:
#!pip install gradio
#!pip install pandas
import gradio as gr
import numpy as np
import pandas as pd
import time

### Settings

In [2]:
# path to excel sheet
excel_path = "study_data.xlsx"

# Videos
#videos = ["testvid1", "testvid2", "testvid3"]
videos = ["video_P09", "video_P09", "video_P09", "video_P09","video_P05",
          "video_P06", "video_P07", "video_P08", "video_P09", "video_P09",
         "video_P11", "video_P12", "video_P13", "video_P09", "video_P15", "testvid1"]
video_type = "mp4"
default_video_path = f"videos/{videos[15]}.{video_type}"
video_format = [380, 540] # height, width

# Images
images = [["E1_ROM_P09_analysis", "E1_COMP_P09_analysis"], 
          ["E1_ROM_P09_analysis", "E1_COMP_P09_analysis"],
          ["E1_ROM_P09_analysis", "E1_COMP_P09_analysis"],
          ["E1_ROM_P09_analysis", "E1_COMP_P09_analysis"],
          ["E1_ROM_P05_analysis", "E1_COMP_P05_analysis"],
          ["E1_ROM_P06_analysis", "E1_COMP_P06_analysis"],
          ["E1_ROM_P07_analysis", "E1_COMP_P07_analysis"],
          ["E1_ROM_P08_analysis", "E1_COMP_P08_analysis"],
          ["E1_ROM_P09_analysis", "E1_COMP_P09_analysis"],
          ["E1_ROM_P09_analysis", "E1_COMP_P09_analysis"],
          ["E1_ROM_P11_analysis", "E1_COMP_P11_analysis"],
          ["E1_ROM_P12_analysis", "E1_COMP_P12_analysis"],
          ["E1_ROM_P13_analysis", "E1_COMP_P13_analysis"],
          ["E1_ROM_P09_analysis", "E1_COMP_P09_analysis"],
          ["E1_ROM_P15_analysis", "E1_COMP_P15_analysis"],
         ]
image_type = "png"
image_path_prefix = "images/base/"
default_image_path = f"images/blank.{image_type}"
image_format = [480, 480] # height, width



# Dropdown options
# options are converted to array indexes for image and videos, 
# option "1" -> videos[0], images[0][0] + images[0][1]
# option "2" -> videos[1], images[1][0] + images[1][1]
options = [["9", "9", "9", "9"], ["5", "6", "12", "13"]] ### baseline

#options = [["9", "9", "9", "9"], ["7", "8", "11", "15"]] ### cfe
'''
options = [["1", "2"]
           , ["3", "4"]
           , ["5", "6"]
           , ["7", "8"]
           , ["9", "10"]
           , ["11", "12"]
           , ["13", "14"]
           , ["15", "16"]
          ]
'''

'\noptions = [["1", "2"]\n           , ["3", "4"]\n           , ["5", "6"]\n           , ["7", "8"]\n           , ["9", "10"]\n           , ["11", "12"]\n           , ["13", "14"]\n           , ["15", "16"]\n          ]\n'

### Functions

In [3]:
def retrieve_input(user, prediction_rom, prediction_comp, w, start_time, selected_patient):
    # save input
    df = pd.read_excel(excel_path, index_col=None)
    time_taken = time.time()-start_time
    mylist = [user, selected_patient.value, w, prediction_rom, prediction_comp, time_taken]
    df.loc[len(df)] = mylist
    df.to_excel(excel_path, index=False)
    
    return f'User number: {user},  ROM Prediction: {prediction_rom}, COMP Prediction: {prediction_comp},  Time taken: {time_taken}s', " "

def start_timer():
    return "Timer Started", time.time()

def select_patient(p):
    patient_num = int(p) - 1
    
    video = videos[patient_num]
    image1 = images[patient_num][0]
    image2 = images[patient_num][1]
        
    return gr.Video.update(value = f"videos/{video}.{video_type}"), gr.Variable(value=f"{p}"), gr.Image.update(f"{image_path_prefix}{image1}.{image_type}"), gr.Image.update(f"{image_path_prefix}{image2}.{image_type}")

def get_usernum(num):
    #ddl = options[int(num)-1]
    ddl = options[int(num)]
    
    return f"User Number: {num} submitted.", num, gr.Dropdown.update(choices = ddl), gr.Dropdown.update(choices = ddl)

### Interface

In [4]:
# Interface
demo = gr.Blocks()

with demo:
    # Welcome Message
    gr.Markdown("Insert Welcome Message Here")
    
    with gr.Tabs():
        usernum_var = gr.Variable()
        
        # Tab 0
        with gr.TabItem("Home Tab"):
            # Enter user number
            usernum = gr.Textbox(placeholder='Enter User Number', label="User Number")
            usernum_btn = gr.Button("Submit User Number")
            usernum_feedback = gr.Markdown()
        
        # Tab 2
        with gr.TabItem("Without Explanations"):
            # Select patient / video
            dropdown2 = gr.Dropdown([""], label="Select Patient")
            selected_patient2 = gr.Variable()
            
            # Start timer
            gr.Markdown("")
            startbtn2 = gr.Button("Start timer")
            start_msg2 = gr.Markdown()
            startbtn_output2 = gr.Variable()
            startbtn2.click(start_timer, inputs=None, outputs=[start_msg2, startbtn_output2])
            
            # Insert video here
            vid2 = gr.Video(default_video_path, format=video_type).style(height=video_format[0], width=video_format[1])
            
            # Dropdown click
            dropdown2.change(select_patient, inputs=dropdown2, outputs=[vid2, selected_patient2])
            
            # Get input
            pred_rom2 = gr.Textbox(placeholder='Enter ROM Assessment', label="ROM Assessment")
            pred_comp2 = gr.Textbox(placeholder='Enter COMP Assessment', label="COMP Assessment")
            output2 = gr.Markdown()
            text_button2 = gr.Button("Submit")
            w2 = gr.Variable(value="No")
            text_button2.click(retrieve_input, inputs=[usernum_var, pred_rom2, pred_comp2, w2, startbtn_output2, selected_patient2]
                              , outputs=[output2, start_msg2])
            
        # Tab 1  
        with gr.TabItem("With Explanations"):
            # Select patient / video
            dropdown = gr.Dropdown([""], label="Select Patient")
            selected_patient = gr.Variable()
            
            # Start timer
            gr.Markdown("")
            startbtn = gr.Button("Start timer")
            start_msg = gr.Markdown()
            startbtn_output = gr.Variable()
            startbtn.click(start_timer, inputs=None, outputs=[start_msg, startbtn_output])
            
            # Insert video here
            vid = gr.Video(default_video_path, format=video_type).style(height=video_format[0], width=video_format[1])
            
            # Insert photos/explanations here
            with gr.Row():
                with gr.Column():
                    ex1 = gr.Image(default_image_path).style(height=image_format[0], width=image_format[1])
                with gr.Column():
                    ex2 = gr.Image(default_image_path).style(height=image_format[0], width=image_format[1])
            
            
            # Dropdown click
            dropdown.change(select_patient, inputs=dropdown, outputs=[vid, selected_patient, ex1, ex2])
            usernum_btn.click(get_usernum, inputs=usernum, outputs=[usernum_feedback, usernum_var, dropdown, dropdown2]) # shifted here
            
            
            # Get input
            pred_rom = gr.Textbox(placeholder='Enter ROM Assessment', label="ROM Assessment")
            pred_comp = gr.Textbox(placeholder='Enter COMP Assessment', label="COMP Assessment")
            output = gr.Markdown()
            text_button = gr.Button("Submit")
            w = gr.Variable(value="Yes")
            text_button.click(retrieve_input, inputs=[usernum_var, pred_rom, pred_comp, w, startbtn_output, selected_patient]
                              , outputs=[output, start_msg])
            
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://17068.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7fb2c9201ca0>,
 'http://127.0.0.1:7860/',
 'https://17068.gradio.app')